In [18]:
import pandas as pd
import numpy as np
import warnings
from sqlalchemy import create_engine
warnings.filterwarnings('ignore')
print('Libraries imported!')

Libraries imported!


In [5]:
df = pd.read_csv("data/calgary_df.csv")
df.head()

,price,address,postal_code,bed,full_bath,half_bath,property_area,property_type
0,299900,2804 - 225 11 AVE SE,T2G 0G3,1.0,1.0,NaN,566.0,condo
1,359900,13 AUBURN BAY GREEN SE,T3M 0A8,2.0,2.0,1.0,1513.0,house
2,899999,1723 10 Street SW,T2T 3E9,4.0,3.0,1.0,2177.0,townhouse
3,259900,315 - 2022 CANYON MEADOWS DR SE,T2J 7H1,2.0,2.0,NaN,1184.0,condo
4,339750,119 SCENIC ACRES TERR NW,T3L 1Y4,3.0,2.0,1.0,1464.0,house


In [38]:
df[['post'][0]] = df["postal_code"].str.split(' ', expand=True)
df.head()

,price,address,postal_code,bed,full_bath,half_bath,property_area,property_type,post
0,299900,2804 - 225 11 AVE SE,T2G 0G3,1.0,1.0,NaN,566.0,condo,T2G
1,359900,13 AUBURN BAY GREEN SE,T3M 0A8,2.0,2.0,1.0,1513.0,house,T3M
2,899999,1723 10 Street SW,T2T 3E9,4.0,3.0,1.0,2177.0,townhouse,T2T
3,259900,315 - 2022 CANYON MEADOWS DR SE,T2J 7H1,2.0,2.0,NaN,1184.0,condo,T2J
4,339750,119 SCENIC ACRES TERR NW,T3L 1Y4,3.0,2.0,1.0,1464.0,house,T3L


In [39]:
df['post'].unique()

array(['T2G', 'T3M', 'T2T', 'T2J', 'T3L', 'T3H', 'T2Z', 'T2X', 'T2L',
       'T3K', 'T2B', 'T3C', 'T2V', 'T3B', 'T2Y', 'T2M', 'T2R', 'T2W',
       'T2A', 'T2C', 'T1Y', 'T3N', 'T3A', 'T3R', 'T3P', 'T2P', 'T2E',
       'T2S', 'T3E', 'T2K', 'T3J', 'T2N', 'T3G', 'T2H', 'O0O', 'T3S', nan,
       'X0X', 'T1X'], dtype=object)

In [63]:
post_group = df.groupby("post")["price"].describe().round()
post_group.head()

,count,mean,std,min,25%,50%,75%,max
post,,,,,,,,
O0O,1.0,334900.0,NaN,334900.0,334900.0,334900.0,334900.0,334900.0
T1X,1.0,1995000.0,NaN,1995000.0,1995000.0,1995000.0,1995000.0,1995000.0
T1Y,58.0,330353.0,102259.0,119900.0,264900.0,340450.0,409800.0,514900.0
T2A,55.0,292396.0,140863.0,25000.0,173944.0,304000.0,369850.0,899000.0
T2B,25.0,281000.0,77715.0,129900.0,225000.0,289000.0,329900.0,469900.0


In [64]:
post_group.reset_index(inplace=True)
post_group.sort_values("mean", ascending=False, inplace=True)
post_group.head()

,post,count,mean,std,min,25%,50%,75%,max
36,T3S,3.0,11666300.0,19774575.0,229000.0,249450.0,269900.0,17384950.0,34500000.0
1,T1X,1.0,1995000.0,NaN,1995000.0,1995000.0,1995000.0,1995000.0,1995000.0
5,T2C,29.0,1265743.0,3252630.0,99900.0,319900.0,409900.0,599900.0,16757850.0
17,T2T,164.0,907561.0,721491.0,125000.0,399900.0,747000.0,1106175.0,4750000.0
18,T2V,79.0,907091.0,1074744.0,23900.0,320500.0,516900.0,899000.0,4999999.0


In [65]:
post_group.dtypes

post      object
count    float64
mean     float64
std      float64
min      float64
25%      float64
50%      float64
75%      float64
max      float64
dtype: object

In [66]:
post_group.drop(["std", "25%", "50%", "75%"], axis=1, inplace=True)
post_group.head()

,post,count,mean,min,max
36,T3S,3.0,11666300.0,229000.0,34500000.0
1,T1X,1.0,1995000.0,1995000.0,1995000.0
5,T2C,29.0,1265743.0,99900.0,16757850.0
17,T2T,164.0,907561.0,125000.0,4750000.0
18,T2V,79.0,907091.0,23900.0,4999999.0


In [67]:
post_group.to_csv('data/post_group.csv', index=False)

In [68]:
rds_connection_string = "postgres:123@localhost:5432/realestate_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

post_group.to_sql(name= "post_group", con=engine, if_exists="replace", index=False)
